In [1]:
import quandl
import pandas as pd
import seaborn as sns
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
import quandl
import seaborn as sns
import sys
import pandas_datareader.data as web
from statsmodels.graphics.tsaplots import plot_acf
import datetime as dt
from dateutil.relativedelta import relativedelta

In [2]:
def Z_score(data):
    # not including the current observation
    std = data.iloc[:-1].std()
    mean = data.iloc[:-1].mean()
    Z = (data[-1:] - mean)/std
    return Z

In [3]:
def color_negative_red(value):
    """
      Colors elements in a dateframe
      green if positive and red if
      negative. Does not color NaN
      values.
      """

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

# Sentiment analysis (Futures only)
- sp500
- gold
- dollar
- Bitcoin (Not option data)


Z score for net positions (1 and 3 years) and change in position (1 month and 3 months).



In [9]:
#data import

gold_tmp = quandl.get("CFTC/088691_F_L_ALL")
sp500_tmp = quandl.get("CFTC/138741_F_L_ALL")
dxy_tmp = quandl.get("CFTC/098662_F_L_ALL")
bitcoin_tmp = quandl.get("CFTC/133741_F_L_ALL")

LimitExceededError: (Status 429) (Quandl Error QELx04) You have exceeded the API speed limit of 20 calls per 10 minutes. Please slow down your requests.

In [18]:
assets = [gold_tmp,sp500_tmp, dxy_tmp, bitcoin_tmp]
asset_names = ['Gold', 'SP500', 'DXY', 'Bitcoin']

In [11]:
for asset in assets:
    asset['Net Position'] = asset['Noncommercial Long'] - asset['Noncommercial Short']
    asset['1 Yr Z Score'] = asset['Net Position'].rolling(52).apply(lambda x: Z_score(x),raw=False)
    asset['3 Yr Z Score'] = asset['Net Position'].rolling(3 * 52).apply(lambda x: Z_score(x),raw=False) 
    asset['1 Month Change'] = asset['Net Position'].diff(4)/asset['Net Position'].shift(4)
    asset['3 Month Change'] = asset['Net Position'].diff(3*4)/asset['Net Position'].shift(3*4)

NameError: name 'assets' is not defined

In [12]:
TOTAL_ASSETS = None

for asset in assets:
    
    TOTAL_ASSETS = pd.concat([TOTAL_ASSETS, asset.tail(1)], axis = 0)
    
TOTAL_ASSETS['Asset'] = asset_names
TOTAL_ASSETS.set_index(['Asset'], inplace=True, drop=True)

compact = ['1 Yr Z Score', '3 Yr Z Score', '1 Month Change', '3 Month Change']
TOTAL_ASSETS = TOTAL_ASSETS[compact]

NameError: name 'assets' is not defined

## Display Results

In [13]:
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]

cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(TOTAL_ASSETS.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(TOTAL_ASSETS.columns), as_cmap=True)

TOTAL_ASSETS_VIEW = (TOTAL_ASSETS.style
    .applymap(color_negative_red, subset=['1 Yr Z Score', '3 Yr Z Score', '1 Month Change', '3 Month Change'])
    #.background_gradient(axis = 0 ,subset = ['DSSP', 'HIDI'],cmap=cm_high_good)
    #.background_gradient(axis = 0 ,subset = ['VRANK', 'MRANK','CRANK', 'ST'],cmap=cm_low_good)
    .set_caption('Speculator Positioning')
    .format({'1 Month Change': "{:.1%}",'3 Month Change': "{:.1%}", '1 Yr Z Score': "{:.2f}", '3 Yr Z Score': "{:.2f}"})
    .set_table_styles(styles))

AttributeError: 'NoneType' object has no attribute 'columns'

# Returns, Realized and Implied Volatility

In [14]:
#Prices

GOLD_price_tmp = yf.download('GLD')
SP500_price_tmp = yf.download('SPY')
DXY_price_tmp = yf.download('DX-Y.NYB')
BIT_price_tmp = yf.download('BTC-USD')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [21]:
GOLD_price = pd.DataFrame(GOLD_price_tmp['Close'])
SP500_price = pd.DataFrame(SP500_price_tmp['Close'])
DXY_price = pd.DataFrame(DXY_price_tmp['Close'])
BIT_price = pd.DataFrame(BIT_price_tmp['Close'])


prices = [GOLD_price,SP500_price,DXY_price, BIT_price ]
price_names = ['Gold', 'SP500', 'DXY', 'Bitcoin']

In [16]:
imp_vol = pd.DataFrame()
imp_vol['Asset'] = ['Gold', 'SP500', 'DXY', 'Bitcoin']
imp_vol['ImpVol'] = np.nan 

tickers = ['GLD', 'SPY', 'UUP']
month_delta = 1

for i in range(len(tickers)):
    try:
        price_ind = yf.download(tickers[i],start='2020-01-01', progress = False, threads = False)
        stock = yf.Ticker(tickers[i])
        option_dates = stock.options
        date = dt.datetime.now()
        date += relativedelta(months=month_delta) 
        expiry_one_month = min(option_dates, key=lambda x:abs(dt.datetime.strptime(x, "%Y-%M-%d")-date))
        current_price = float(price_ind['Close'].tail(1))
        
        calls = stock.option_chain(expiry_one_month).calls
        puts = stock.option_chain(expiry_one_month).puts
        
        #ATM Skew
        ATM_put = puts[puts['strike'] == min(puts['strike'], key=lambda x:abs(x-current_price))]
        ATM_call = calls[calls['strike'] == min(calls['strike'], key=lambda x:abs(x-current_price))]
        ATM_skew = float(ATM_put['impliedVolatility']) - float(ATM_call['impliedVolatility']) 
        
        imp_vol.loc[i,"ImpVol"] = float(ATM_put['impliedVolatility'])
        
    except:
        print("ERROR! INDEX: ", i, ', TICKER: ', tickers[i])



In [19]:
for price in prices:
    price['1 Month Return'] = price['Close'].pct_change(20)
    price['3 Month Return'] = price['Close'].pct_change(60)
    price['Volatility'] = price['Close'].pct_change().rolling(21).std()*(252**0.5)
    price['3 year Volatility Z-Score'] = price['Volatility'].rolling(3 * 52).apply(lambda x: Z_score(x),raw=False) 


PRICES = None

for price in prices:
    
    PRICES = pd.concat([PRICES, price.tail(1)], axis = 0)
    
PRICES['Asset'] = asset_names
compact_1 = ['Asset', '1 Month Return', '3 Month Return', 'Volatility', '3 year Volatility Z-Score']
PRICES = PRICES[compact_1]
PRICES.set_index(['Asset'], inplace=True, drop=True)

PRICES = pd.merge(PRICES, imp_vol, how="left", on=['Asset'])
PRICES['Implied Vol Premium'] = PRICES['ImpVol'] - PRICES['Volatility']

In [ ]:
th_props = [
  ('font-size', '16px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', 'Black'),
  ('background-color', '#f7f7f9')
  ]


# Set CSS properties for td elements in dataframe
td_props = [('font-size', '16px'), ('text-align', 'center')]

# Set table styles
styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props)
]


cm_low_good = sns.diverging_palette(150, 10, s=80, l=70, n=len(PRICES.columns), as_cmap=True)#sns.light_palette("green", as_cmap=True)
cm_high_good = sns.diverging_palette(10, 150, s=80, l=70, n=len(PRICES.columns), as_cmap=True)

PRICES_VIEW = (PRICES.style
    .applymap(color_negative_red, subset=['1 Month Return', '3 Month Return', 'Volatility', '3 year Volatility Z-Score', 'ImpVol', 'Implied Vol Premium'])
    .set_caption('Asset Metrics')
    .format({'1 Month Return': "{:.1%}",'3 Month Return': "{:.1%}", 'Volatility': "{:.1%}", '3 year Volatility Z-Score': "{:.2f}", 
            'ImpVol': "{:.1%}",'Implied Vol Premium': "{:.1%}" })
    .set_table_styles(styles))

## View metrics

In [ ]:
PRICES_VIEW

In [ ]:
TOTAL_ASSETS_VIEW